In [1]:
# Внимание!!! Важно, что бы файлы с данными и исполняемый файл находились в одной папке, 
# тогда пути к тестовым и тренировочным наборам будут содержать только имена файлов.
# 
# В пути к тренировочным и тестовым данным запрежается использовать абсалютную адресацию, 
# то есть адресацию, в которой присутствуют имена папок. Путь должен содержать только имя файла.
#
# Напоминание: под моделью машинного обучения понимаются все действия с исходными данными, 
# которые необходимо произвести, что бы сопоставить признаки целевому значению.

### Область работы 1 (библиотеки)

In [24]:
# Данный блок в области 1 выполняется преподавателем
# 
# данный блок предназначен только для подключения необходимых библиотек
# запрещается подключать библиотеки в других блоках
#
# установка дополнительных библиотек размещается прямо здесь (обязательно закоментированы)
#
# pip install

In [26]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from IPython.display import display

from sklearn.pipeline import make_pipeline, Pipeline

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.preprocessing import RobustScaler, Normalizer

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import GridSearchCV, KFold, StratifiedShuffleSplit, StratifiedKFold, RepeatedKFold

from sklearn.neighbors import KNeighborsRegressor

import sklearn

import pandas.plotting as pd_plt
import matplotlib.colors as plt_colors

from sklearn import svm

from sklearn.model_selection import cross_validate
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer , make_column_transformer, TransformedTargetRegressor
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, KNNImputer, IterativeImputer

import category_encoders as ce
from sklearn.preprocessing import PolynomialFeatures, QuantileTransformer, PowerTransformer
from sklearn.linear_model import ElasticNet, Ridge, LinearRegression, Lasso, Lars
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.svm import SVR
from scipy.stats import boxcox
from scipy.special import inv_boxcox
from scipy import stats

from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_squared_log_error, r2_score, mean_absolute_percentage_error

from sklearn.linear_model import ElasticNet, Ridge, Lasso, Lars,SGDRegressor,LogisticRegression, LinearRegression

from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor,VotingRegressor

### Область работы 2 (выполнение лучшей модели)

In [30]:
# Данный блок(и) в области 2 выполняется преподавателем
#
# В области находится одна, единственная, итоговая модель машинного обучения с однозначными, 
# зафиксированными параметрами
#
# В данной области категорически запрещается искать, выбирать, улучшать, оптимизировать, 
# тюниговать и т.д. модель машинного обучения

In [32]:
# Путь к тренировочному набору
path_train = 'train.csv' # содержит только имя файла, без имен папок
# Путь к тестовому набору
path_test  = 'test.csv' # содержит только имя файла, без имен папок

In [34]:
df = pd.read_csv(path_train)
X_test = pd.read_csv(path_test)

In [36]:
df.drop_duplicates(inplace=True)

In [38]:
X_train = df.drop(['Weight'], axis = 1)
y_train = df['Weight']

In [40]:
age = ['Age']
num_f = ['Length', 'Girth', 'Height']
num = ['BCS']
sex = ['Sex']

In [42]:
age_map = [{
    'col':'Age',
    'mapping':{'>20':5, '15-20':4, '10-15':3, '5-10':2, '2-5':1, '<2':0}
    }]

In [44]:
transformer_age = Pipeline(steps=[
    ('ce',ce.OrdinalEncoder(mapping=age_map)),
    ('scaler', StandardScaler())
])

transformer_sex = Pipeline(steps=[
    ('onehot', OneHotEncoder(sparse_output=False, handle_unknown='ignore').set_output(transform='pandas')),
    ('scaler', StandardScaler())
])

transformer_num_f = Pipeline(steps=[
    ('polynom', PolynomialFeatures(3,include_bias=False)),
    ('scaler', StandardScaler())
])

transformer_num = Pipeline(steps=[
    ('scaler', StandardScaler())
])

In [46]:
CT = ColumnTransformer([
        ("age", transformer_age, age),
        ("num", transformer_num, num),
        ("sex", transformer_sex, sex),
        ("num_f", transformer_num_f, num_f)
        ]).set_output(transform="pandas")

display(CT)

ColumnTransformer(transformers=[('age',
                                 Pipeline(steps=[('ce',
                                                  OrdinalEncoder(mapping=[{'col': 'Age',
                                                                           'data_type': dtype('O'),
                                                                           'mapping': >20      5
15-20    4
10-15    3
5-10     2
2-5      1
<2       0
dtype: int64}])),
                                                 ('scaler', StandardScaler())]),
                                 ['Age']),
                                ('num',
                                 Pipeline(steps=[('scaler', StandardScaler())]),
                                 ['BCS']),
                                ('sex',
                                 Pipeline(steps=[('onehot',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False)),
                                                 ('scaler', StandardScaler())]),
                                 ['Sex']),
                                ('num_f',
                                 Pipeline(steps=[('polynom',
                                                  PolynomialFeatures(degree=3,
                                                                     include_bias=False)),
                                                 ('scaler', StandardScaler())]),
                                 ['Length', 'Girth', 'Height'])])

In [48]:
# Блок обучения модели

In [50]:
model = Pipeline([('preprocessing', CT), ('regressor', GradientBoostingRegressor(learning_rate=0.01,
                                                                              subsample=0.1,
                                                                              min_samples_split = 9,
                                                                              max_depth = 4,
                                                                              n_estimators=500,
                                                                              random_state=52))])
model.fit(X_train, y_train)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('age',
                                                  Pipeline(steps=[('ce',
                                                                   OrdinalEncoder(mapping=[{'col': 'Age',
                                                                                            'data_type': dtype('O'),
                                                                                            'mapping': >20      5
15-20    4
10-15    3
5-10     2
2-5      1
<2       0
dtype: int64}])),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['Age']),
                                                 ('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['BCS']),
                                                 ('sex',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEn...',
                                                                                 sparse_output=False)),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['Sex']),
                                                 ('num_f',
                                                  Pipeline(steps=[('polynom',
                                                                   PolynomialFeatures(degree=3,
                                                                                      include_bias=False)),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['Length', 'Girth',
                                                   'Height'])])),
                ('regressor',
                 GradientBoostingRegressor(learning_rate=0.01, max_depth=4,
                                           min_samples_split=9,
                                           n_estimators=500, random_state=52,
                                           subsample=0.1))])

In [52]:
# Блок предсказания с использованием тестового набора

In [54]:
y_pred = model.predict(X_test)

In [56]:
# Вектора предсказанных значений  y_predict полученый на основане тестового набора
y_predict = y_pred